In [197]:
import pickle
import numpy as np
import pandas as pd

def load_tree(input_file):
    with open(input_file, 'rb') as f:
        return pickle.load(f)

tree = load_tree('sample_data/combined_dendrogram.pkl')

embeddings = np.load('sample_data/combined_reduced_embeddings.npz')
filenames = pd.read_csv('sample_data/combined_filenames.csv', header=None).values.flatten()

In [198]:
# lists contents of embeddings
embeddings.files

['pca5', 'tsne2', 'umap5', 'umap2']

In [199]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import pyvista as pv
from shapely import Point, MultiPoint, box, voronoi_polygons, convex_hull
import matplotlib.colors as mcolors
from pyvista import examples

def gdf_to_flat_mesh(gdf, cluster_rgb):
    
    gdf['area'] = gdf['geometry'].area
    gdf = gdf.sort_values('area').reset_index(drop=True)
    labels = gdf['filename']

    vertices = []
    faces = []
    
    poly_ids = []
    face_labels = []
    face_colors = []

    for index, row in gdf.iterrows():
        polygon = row['geometry']
        if not polygon.is_valid:
            continue

        exterior = np.array(polygon.exterior.coords)
        exterior_3d = np.hstack([exterior, np.zeros((len(exterior), 1))]) 
        vertices.append(exterior_3d)

        face = np.arange(len(exterior), dtype=np.int64) + len(np.vstack(vertices)[:-len(exterior)])
        faces.append(face)

        # Assign an ID to each polygon
        poly_ids.extend([index] * len(face))

        
        face_label = labels[index]
        face_labels.append(face_label)

        face_color = cluster_rgb
        face_colors.append(face_color)

    vertices = np.vstack(vertices)
    faces = np.hstack([np.hstack([[len(face)], face]) for face in faces])

    mesh = pv.PolyData(vertices, faces)
   
    labels_encoded = [s.encode('utf-8') for s in face_labels]

    
    mesh["PolyIDs"] = -np.array(poly_ids) 
    mesh["FaceLabels"] = labels_encoded
    mesh["FaceColors"] = face_colors
    

    return mesh

def create_voronoi_gdf(polygons, edges, filenames):
    voronoi_polygons = []
    voronoi_filenames = []

    for i, polygon in enumerate(polygons.geoms):
        clipped_polygon = polygon.intersection(edges)
        voronoi_polygons.append(clipped_polygon)
        voronoi_filenames.append(filenames[i])

    gdf = gpd.GeoDataFrame(geometry=voronoi_polygons)
    gdf['filename'] = voronoi_filenames  
    return gdf


def get_rgb(color_string):
    rgb = mcolors.to_rgb(color_string)
    
    rgb = tuple([int(x*255) for x in rgb])
    return rgb



In [200]:
dim_reduction_values = {
    "pca5": [0.0002, 4.5],
    "umap2": [0.003, 4.5],
    "tsne2": [0.01, 15.5],
    "umap5": [0.0002, 4.5]
}

def flat_mesh_to_terrain(mesh, dim_reduction='umap5'):
    if dim_reduction not in dim_reduction_values:
        raise ValueError(f"Unknown dim_reduction value: {dim_reduction}")
        
    factor, alpha = dim_reduction_values[dim_reduction]
    
    warp = mesh.warp_by_scalar(factor=factor)
    surf = warp.delaunay_2d(alpha=alpha)
    surf_smoothed = surf.smooth(n_iter=150)
    
    return surf_smoothed

from scipy.interpolate import griddata

def interpolate_z(file_points, terrain):
    file_points_coords = file_points.points
    terrain_coords = terrain.points

    # Use only X and Y for interpolation
    points_xy = file_points_coords[:, :2]
    terrain_xy = terrain_coords[:, :2]

    # Interpolate Z values from terrain onto point positions
    interpolated_z = griddata(terrain_xy, terrain_coords[:, 2], points_xy)

    # Create a new set of points with the interpolated Z values
    new_points = np.hstack((points_xy, interpolated_z.reshape(-1, 1)))

    file_points.points = new_points
    return file_points

def file_coord_to_dataframe(points, labels, csv_file='test_dataframe.csv'):
    
    file_coords = points.points

    df = pd.DataFrame(file_coords, columns=['X', 'Y', 'Z'])
    df['FaceLabels'] = labels

    df.to_csv(csv_file, index=False)

In [201]:
def visualize_clusters(tree, level, points):
    clusters = get_clusters(tree, level)
    meshs = list()
    cluster_file_points = list()
    cluster_labels = list()
   
    colors = ["red", "green", "blue", "yellow", "purple"]
    
    for i, cluster in enumerate(clusters):
        indices = [np.where(filenames == fname)[0][0] for fname in cluster]

        color = colors[i % len(colors)]        
        cluster_points = [Point(x, y) for x, y in points[indices]]
        cluster_color = get_rgb(color)
            
        cluster_points_shaply_obj = MultiPoint(cluster_points)
        hull = MultiPoint(cluster_points).convex_hull
        polygons = voronoi_polygons(cluster_points_shaply_obj)
        
        gdf = create_voronoi_gdf(polygons, hull, filenames)

        mesh = gdf_to_flat_mesh(gdf, cluster_color)
        surf = flat_mesh_to_terrain(mesh, 'pca5')

        centers = mesh.cell_centers()
        file_coord = interpolate_z(centers, surf)
        labels = mesh['FaceLabels']
        
        meshs.append(surf)   
        cluster_file_points.append(file_coord)
        cluster_labels.append(labels)
        
    return meshs, cluster_file_points, cluster_labels

In [202]:
pca5_points = embeddings['pca5'][:, :2]
umap2_points = embeddings['umap2']
umap5_points = embeddings['umap5'][:, :2]
tsne2_points = embeddings['tsne2']

for level in range(1, 4):
    meshes, label_coord, labels = visualize_clusters(tree, level, pca5_points)

volumes = [mesh.volume for mesh in meshes]
sorted_indices = np.argsort(volumes)
meshes_sorted = [meshes[i] for i in sorted_indices]

# translation distance based on largest mesh
translation_distance = meshes_sorted[-1].length 


pl = pv.Plotter()

translations = np.array([
    [0, -translation_distance, 0],  # Down
    [-translation_distance, -translation_distance, 0],  # Left-Down
    [-translation_distance, 0, 0],  # Left
    [-translation_distance, translation_distance, 0],  # Left-Up
    [0, translation_distance, 0],  # Up
    [translation_distance, translation_distance, 0],  # Right-Up
    [translation_distance, 0, 0],  # Right
    [translation_distance, -translation_distance, 0],  # Right-Down
])


for i in range(7):
    meshes[i].points += translations[i]
    label_coord[i].points += translations[i]


for i, mesh in enumerate(meshes):
    
    pl.add_mesh(mesh)
    pl.add_point_labels(label_coord[i], labels[i],  point_size=5, font_size=10)
    
    #filename_path= f"labels_coord/umap5_cluster_{i}.csv"
    #file_coord_to_dataframe(label_coord[i], labels[i], filename_path)
    
#pl.export_obj('meshes/pca_clusters.obj') # comment out pl.add_point_labels() to save the mesh only

pl.show()

Widget(value="<iframe src='http://localhost:56361/index.html?ui=P_0x24ee0969660_102&reconnect=auto' style='wid…